In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

print(project_root)

/workspace/dna-neural-network-coi-


In [2]:
import torch
print(f"¿GPU disponible? {torch.cuda.is_available()}")
print(f"Número de GPUs: {torch.cuda.device_count()}")
print(f"Nombre GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

¿GPU disponible? False
Número de GPUs: 0
Nombre GPU: No GPU


/venv/main/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/venv/main/lib/python3.12/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import json

# Tus imports
from src.combined_model.combined_model_embedding import *
from src.combined_model.combined_models import *
from src.decoders.decoder_simple import *
from src.encoders_model.DNABERT_Embedder import *
from src.encoders_model.embdeeding_encoders import *
from src.encoders_model.simple_encoders import *
from src.evaluators.linear_evaluator import *
from src.decoders.sequence_decoder import *


In [4]:
from src.encoders_model.simple_encoders import *
from src.utils.load_fastaDataset import *
from src.training.experimentRunner import *
from torch.utils.data import DataLoader

In [5]:
def load_hierarchy_from_json(json_path):
    """Carga la jerarquía taxonómica desde JSON"""
    with open(json_path, 'r') as f:
        hierarchy_raw = json.load(f)
    
    # Convertir claves string a int
    hierarchy = {}
    for child_taxon, parent_dict in hierarchy_raw.items():
        hierarchy[child_taxon] = {}
        for parent_key, children_list in parent_dict.items():
            parent_int = int(float(parent_key))
            children_int = [int(c) for c in children_list]
            hierarchy[child_taxon][parent_int] = children_int
    
    print("✅ Jerarquía cargada desde JSON")
    for taxon, mapping in hierarchy.items():
        n_parents = len(mapping)
        n_children = sum(len(v) for v in mapping.values())
        print(f"  {taxon:10s}: {n_parents:4d} padres → {n_children:5d} hijos")
    
    return hierarchy

In [6]:
hierarchy_path = os.path.join(project_root, "src", "data", "taxonomy_hierarchy_fixed_with_class.json")
hierarchy = load_hierarchy_from_json(hierarchy_path)

✅ Jerarquía cargada desde JSON
  class     :   49 padres →   187 hijos
  order     :  173 padres →   831 hijos
  family    :  797 padres →  5446 hijos
  genus     : 5393 padres → 50568 hijos
  species   : 50510 padres → 205075 hijos


In [7]:
df = os.path.join(project_root, "src", "data", "all_taxa_numeric.csv")
df = pd.read_csv(df)

In [8]:
taxon_order = ['phylum', 'class','order', 'family', 'genus', 'species']
total_classes = {}
for taxon in taxon_order:
    n_classes = df[taxon].nunique()
    total_classes[taxon] = n_classes
    print(f"  {taxon:10s}: {n_classes:6d} clases")

  phylum    :     49 clases
  class     :    173 clases
  order     :    797 clases
  family    :   5393 clases
  genus     :  50510 clases
  species   : 205075 clases


In [9]:
from sklearn.model_selection import train_test_split

# Primero separar test (20%)
df_temp, df_test = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['phylum']
)

# Luego separar train/val (80/20 del 80% restante = 64/16 del total)
df_train, df_val = train_test_split(
    df_temp, test_size=0.2, random_state=42, stratify=df_temp['phylum']
)

In [10]:
max_length = 750

train_dataset = MultiTaxaFastaDataset(
    df_train.reset_index(drop=True), 
    max_length=max_length,
    taxon_cols=taxon_order
)

val_dataset = MultiTaxaFastaDataset(
    df_val.reset_index(drop=True),
    max_length=max_length,
    taxon_cols=taxon_order
)

test_dataset = MultiTaxaFastaDataset(
    df_test.reset_index(drop=True),
    max_length=max_length,
    taxon_cols=taxon_order
)

In [11]:
def collate_multitask(batch, taxon_cols=['phylum', 'class','order','family','genus','species'], max_length=900):
    sequences, labels_dict_list, recon_targets_list, true_tokens_list = zip(*batch)

    # Labels: dict de tensors
    labels_dict = {taxon: torch.stack([d[taxon] for d in labels_dict_list]) for taxon in taxon_cols}

    # Recon targets: dict de tensors
    recon_targets_dict = {taxon: torch.stack([d[taxon] for d in recon_targets_list]) for taxon in taxon_cols}

    # True tokens
    true_tokens = torch.stack(true_tokens_list)

    return sequences, labels_dict, recon_targets_dict, true_tokens


In [12]:


train_loader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    drop_last=True,
    collate_fn=lambda b: collate_multitask(b, taxon_cols=val_dataset.taxon_cols, max_length=val_dataset.max_length),
    num_workers=8
)



val_loader = DataLoader(
    val_dataset,
    batch_size=256,
    shuffle=False,
    drop_last=True,
    collate_fn=lambda b: collate_multitask(b, taxon_cols=val_dataset.taxon_cols, max_length=val_dataset.max_length),
    num_workers=8
)

test_loader  = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    drop_last=True,
    collate_fn=lambda b: collate_multitask(b, taxon_cols=val_dataset.taxon_cols, max_length=val_dataset.max_length),
    num_workers=8
)


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  Dispositivo: {device}")

# DNABERT
dnabert_path = os.path.join(project_root, "src", "data", "archives")
dnabert = DNABERTEmbedder(
    model_name=dnabert_path,
    max_length=max_length,
    device=device
)
embed_dim = dnabert.get_embedding_dim()

# Encoder
latent_dim = 256
encoder = SimpleEmbeddingEncoder(
    embed_dim=embed_dim,
    latent_dim=latent_dim,
    dropout=0.1
)

# Decoders (uno por taxón)
decoders_dict = {}
for taxon in taxon_order:
    decoders_dict[taxon] = SequenceDecoder(
        latent_dim=latent_dim,
        seq_len=max_length,
        vocab_size=4,
        dropout=0.1
    )

# Global decoder
global_decoder = SequenceDecoder(
    latent_dim=latent_dim,
    seq_len=max_length,
    vocab_size=4,
    dropout=0.1
)

# Classifiers (uno por taxón)
classifiers_dict = {}
print(f"\n🔧 Creando classifiers con número TOTAL de clases:")
for taxon in taxon_order:
    # ✅ CORRECTO: Usar total_classes (del dataset completo)
    # ❌ INCORRECTO: n_classes = df_train[taxon].nunique()
    n_classes = total_classes[taxon]
    
    classifiers_dict[taxon] = CosineClassifier(
        latent_dim=latent_dim,
        num_classes=n_classes,
        scale=20.0  # ⭐ Ajustable si es necesario
    )
    print(f"  {taxon:10s}: {n_classes:6d} clases")

# ⭐ CREAR MODELO JERÁRQUICO
model = HierarchicalCombinedModelFixed(
    dnabert=dnabert,
    encoder=encoder,
    decoders_dict=decoders_dict,
    classifiers_dict=classifiers_dict,
    global_decoder=global_decoder,
    taxonomy_hierarchy=hierarchy  # ⭐ Aquí usas la jerarquía
)

print(f"\n✅ Modelo jerárquico creado")

Some weights of BertModel were not initialized from the model checkpoint at /workspace/dna-neural-network-coi-/src/data/archives and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🖥️  Dispositivo: cpu

🔧 Creando classifiers con número TOTAL de clases:
  phylum    :     49 clases
  class     :    173 clases
  order     :    797 clases
  family    :   5393 clases
  genus     :  50510 clases
  species   : 205075 clases
✅ Modelo jerárquico creado con máscaras suaves (no -inf)

✅ Modelo jerárquico creado


In [14]:
runner = ExperimentRunner(model, device=device)

TARGET_ACCURACY = 0.85  # 85% para avanzar al siguiente nivel
MAX_EPOCHS_PER_LEVEL = 2  # Máximo 3 épocas por nivel
TOTAL_EPOCHS = 12  # Total de épocas


curriculum_trainer = AdaptiveCurriculumTrainer(
    model=model,
    runner=runner,
    device=device,
    target_accuracy=TARGET_ACCURACY,
    max_epochs_per_level=MAX_EPOCHS_PER_LEVEL
)

→ Modo MULTITASK ACTIVADO (clasificación + reconstrucción)
🎓 Curriculum Learning Adaptativo Inicializado
   Target accuracy: 85%
   Max epochs por nivel: 2


In [15]:
history = curriculum_trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    total_epochs=8,  
    alpha=1.0,
    beta=1.0,
    lr=1e-3,
    mixed_precision=True,
    enable_early_stop=True  
)

/venv/main/lib/python3.12/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(



🎓 INICIANDO CURRICULUM LEARNING ADAPTATIVO
🎯 Threshold objetivo: 85.0%
⏱️  Épocas máximas por nivel: 2
📊 Épocas máximas totales: 8
🛑 Early stopping: Activado

EPOCH 1/8
📚 Nivel actual: 1/6 - phylum
   Épocas en este nivel: 1/2
   Pesos: {'phylum': 5.0, 'class': 1.0, 'order': 1.0, 'family': 1.0, 'genus': 1.0, 'species': 1.0}
📊 Pesos por taxón: {'phylum': 5.0, 'class': 1.0, 'order': 1.0, 'family': 1.0, 'genus': 1.0, 'species': 1.0}
🚀 Mixed Precision: True
📦 Batches por época: 3861


[TRAIN] Epoch 1/1:   0%|                                                                                 | 0/3861 [00:00<?, ?it/s]/venv/main/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
[TRAIN] Epoch 1/1:   0%|                                                                                 | 0/3861 [00:00<?, ?it/s]


AssertionError: 

In [ ]:
test_results = runner.evaluate_multitask(
    test_loader=test_loader,
    heads=taxon_order
)

In [ ]:
print(f"\n📊 Resultados finales en test:")
print(f"{'='*70}")
for taxon in taxon_order:
    acc = test_results[taxon]['acc']
    f1 = test_results[taxon]['f1']
    print(f"{taxon:12s} → Acc: {acc:6.2%} | F1: {f1:.4f}")
print(f"{'='*70}\n")

print("\n✅ Script completado")

In [ ]:
torch.save({
    'epoch': 10,  # Última época completada
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'history': history,
    'config': {
        'latent_dim': 128,
        'max_length': 900,
        'taxon_cols': taxon_cols,
        'num_classes': {taxon: df[taxon].nunique() for taxon in taxon_cols}
    }
}, 'checkpoint_epoch10_completo.pt')

print("✅ Checkpoint completo guardado")

In [ ]:
# ===== GUARDAR MODELO COMPLETO =====
import torch
import pickle

# Después de terminar el entrenamiento
checkpoint = {
    'epoch': 10,  # o las épocas que completaste
    'model_state_dict': model.state_dict(),
    'history': history,
    'config': {
        'latent_dim': 128,
        'max_length': 900,
        'taxon_cols': taxon_cols,
        'num_classes': {taxon: df[taxon].nunique() for taxon in taxon_cols},
        'dnabert_path': DNABERT_MODEL
    },
    # Opcional: guardar mapeos label→nombre
    'label_mappings': {
        taxon: {idx: name for idx, name in enumerate(sorted(df[taxon].unique()))}
        for taxon in taxon_cols
    }
}

torch.save(checkpoint, 'modelo_coi_10epochs.pt')
print("✅ Modelo guardado en 'modelo_coi_10epochs.pt'")

# También guardar historial separado para análisis
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history, f)